# Лабораторная работа №2

Требования:
* Python >= 3.X
* NLTK >= 3.2.5

Лабораторную работу необходимо выполнять в данном шаблоне. Результат работы выслать письмом на litvinov.vg@ssau.ru. В теме письма указывать ФИО полностью.

Необходимо провести исследование различных способов представления документов и их влияние на качество определения тональности.

В качестве входных данных к лабораторной работе взят широко известный набор данных IMDB, содержащий 50K обзоров фильмов ([imdb-dataset-of-50k-movie-reviews](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews)). Откликами являются значения двух классов positive и negative.

In [47]:
import re
import nltk
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter

from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score 
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV

In [2]:
data = pd.read_csv(r"data\IMDB Dataset.csv")
print(len(data))
data.head()

50000


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


### Шаг 1. Подготовка данных

В качестве исследуемых способов представления необходимо рассмотреть:
#### 1. Bag of words: word counts ([CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)). Компоненты вектора: частоты или относительные частоты.

In [3]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['review'])
X.shape

(50000, 101895)

In [ ]:
# vectorizer1 = CountVectorizer()
# X1 = vectorizer1.fit_transform(data['review']).toarray()
# X1_train, X1_test, y_train, y_test = train_test_split(X1, y, test_size=0.3, random_state=63)

#### 2. Bag of words: weird numbers ([TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)). Компоненты вектора: оценки "значимости" слова (например tf-idf).

In [4]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['review'])
X.shape

(50000, 101895)

In [ ]:
# vectorizer2 = TfidfVectorizer()
# X2 = vectorizer2.fit_transform(data['review']).toarray()
# X2_train, X2_test = train_test_split(X2, test_size=0.3, random_state=63)

#### 3. Bag of ngrams. Позволяет учитывать положение слов. Компоненты вектора: частоты N-грам. Примитивный подход, но в некоторых задачах он может улучшать качество решения.

In [9]:
cleaner = lambda text: re.sub('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});','',text)
data['review'] = data['review'].map(cleaner)

In [10]:
bag_of_ngrams = data['review'].map(lambda text: tuple(nltk.ngrams(text.split(' '), 2)))

In [11]:
le = LabelEncoder()
data['sentiment'] = le.fit_transform(data['sentiment'])
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. The filming tec...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [45]:
bag_of_ngrams[:10]

0    ((One, of), (of, the), (the, other), (other, r...
1    ((A, wonderful), (wonderful, little), (little,...
2    ((I, thought), (thought, this), (this, was), (...
3    ((Basically, there's), (there's, a), (a, famil...
4    ((Petter, Mattei's), (Mattei's, "Love), ("Love...
5    ((Probably, my), (my, all-time), (all-time, fa...
6    ((I, sure), (sure, would), (would, like), (lik...
7    ((This, show), (show, was), (was, an), (an, am...
8    ((Encouraged, by), (by, the), (the, positive),...
9    ((If, you), (you, like), (like, original), (or...
Name: review, dtype: object

### Шаг 2. Исследование моделей

В зависимости от способа представления оценить качество классификации как долю правильных ответов на выборке (accuracy). Не забывайте использовать перекрестную проверку ([cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html), [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)).

Для каждого из нижеперечисленных моделей необходимо определить оптимальные гиперпараметры ([GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html))

In [16]:
X = data['review']
y = data['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [17]:
count_vec = CountVectorizer()
countvec_train = count_vec.fit_transform(X_train)
countvec_test = count_vec.transform(X_test)

In [18]:
tfidf_vec = TfidfVectorizer()
tfidfvec_train = tfidf_vec.fit_transform(X_train)
tfidfvec_test = tfidf_vec.transform(X_test)

Качество классификации оцениваем для следующих моделей:

#### 1. Машина опорных векторов ([SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)).

##### Count vectorizer

In [26]:
%%time

parameters = {'C':[1, 5, 10]}
linear_svc = LinearSVC(random_state=63)
cv = KFold(n_splits=10, shuffle=True, random_state=63)

svm_clf_count = GridSearchCV(linear_svc, parameters, cv=cv)
svm_clf_count.fit(countvec_train, y_train)

Wall time: 8min 6s


GridSearchCV(cv=KFold(n_splits=10, random_state=63, shuffle=True),
             estimator=LinearSVC(random_state=63),
             param_grid={'C': [1, 5, 10]})

##### TF-iDF

In [34]:
%%time

parameters = {'C':[1, 5, 10]}
linear_svc = LinearSVC(random_state=63)
cv = KFold(n_splits=5, shuffle=True, random_state=63)

svm_clf_tfidf = GridSearchCV(linear_svc, parameters, cv=cv)
svm_clf_tfidf.fit(tfidfvec_train, y_train)

Wall time: 40.4 s


GridSearchCV(cv=KFold(n_splits=5, random_state=63, shuffle=True),
             estimator=LinearSVC(random_state=63),
             param_grid={'C': [1, 5, 10]})

#### 2. Случайный лес ([RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)).

##### Count vectorizer

In [37]:
%%time

parameters = {'n_estimators': [5, 10, 50]}
forest = RandomForestClassifier(random_state=63)
cv = KFold(n_splits=5, shuffle=True, random_state=63)

forest_clf_count = GridSearchCV(forest, parameters, cv=cv)
forest_clf_count.fit(countvec_train, y_train)

Wall time: 15min 31s


GridSearchCV(cv=KFold(n_splits=5, random_state=63, shuffle=True),
             estimator=RandomForestClassifier(random_state=63),
             param_grid={'n_estimators': [5, 10, 50]})

##### TF-iDF

In [39]:
%%time

parameters = {'n_estimators': [5, 10, 50]}
forest = RandomForestClassifier(random_state=63)
cv = KFold(n_splits=5, shuffle=True, random_state=63)

forest_clf_tfidf = GridSearchCV(forest, parameters, cv=cv)
forest_clf_tfidf.fit(tfidfvec_train, y_train)

Wall time: 20min 15s


GridSearchCV(cv=KFold(n_splits=5, random_state=63, shuffle=True),
             estimator=RandomForestClassifier(random_state=63),
             param_grid={'n_estimators': [5, 10, 50]})

### Шаг 3. Сравнение результатов

Сравнить точность обученных моделей. Найти наиболее точную.

In [32]:
def evaluator(estimator, x_test, y_test):
    predict = estimator.predict(x_test)
    print("Classification report: \n", classification_report(y_test, predict, target_names=['Negative','Positive']))
    print("Confusion Matrix: \n", confusion_matrix(y_test, predict))
    print("Accuracy: \n", accuracy_score(y_test, predict))

In [41]:
# SVC CountVectorizer
evaluator(svm_clf_count.best_estimator_, countvec_test, y_test)

Classification report: 
               precision    recall  f1-score   support

    Negative       0.87      0.86      0.86      7484
    Positive       0.86      0.87      0.87      7516

    accuracy                           0.87     15000
   macro avg       0.87      0.87      0.87     15000
weighted avg       0.87      0.87      0.87     15000

Confusion Matrix: 
 [[6442 1042]
 [ 976 6540]]
Accuracy: 
 0.8654666666666667


In [42]:
# SVC TF-iDF
evaluator(svm_clf_tfidf.best_estimator_, tfidfvec_test, y_test)

Classification report: 
               precision    recall  f1-score   support

    Negative       0.90      0.89      0.90      7484
    Positive       0.89      0.91      0.90      7516

    accuracy                           0.90     15000
   macro avg       0.90      0.90      0.90     15000
weighted avg       0.90      0.90      0.90     15000

Confusion Matrix: 
 [[6656  828]
 [ 710 6806]]
Accuracy: 
 0.8974666666666666


In [43]:
# RandomForestClassifier CountVectorizer
evaluator(forest_clf_count.best_estimator_, countvec_test, y_test)

Classification report: 
               precision    recall  f1-score   support

    Negative       0.82      0.84      0.83      7484
    Positive       0.84      0.82      0.83      7516

    accuracy                           0.83     15000
   macro avg       0.83      0.83      0.83     15000
weighted avg       0.83      0.83      0.83     15000

Confusion Matrix: 
 [[6281 1203]
 [1342 6174]]
Accuracy: 
 0.8303333333333334


In [44]:
# RandomForestClassifier TF-iDF
evaluator(forest_clf_tfidf.best_estimator_, tfidfvec_test, y_test)

Classification report: 
               precision    recall  f1-score   support

    Negative       0.81      0.83      0.82      7484
    Positive       0.83      0.80      0.82      7516

    accuracy                           0.82     15000
   macro avg       0.82      0.82      0.82     15000
weighted avg       0.82      0.82      0.82     15000

Confusion Matrix: 
 [[6248 1236]
 [1466 6050]]
Accuracy: 
 0.8198666666666666


### Вывод

- Наибольшее значение **Precision** ``0.90`` показал **SVM** классификатор с **TF-iDF** препроцессингом
- Наибольшее значение **Recall** ``0.90`` показал **SVM** классификатор с **TF-iDF** препроцессингом
- Наибольшее значение **F1-score** ``0.90`` показал **SVM** классификатор с **TF-iDF** препроцессингом
- Наибольшее значение **Accuracy** ``0.8974666666666666`` показал **SVM** классификатор с **TF-iDF** препроцессингом
- Наименьшее значение **Precision** ``0.82`` показал **RandomForestClassifier** классификатор с **TF-iDF** препроцессингом
- Наименьшее значение **Recall** ``0.82`` показал **RandomForestClassifier** классификатор с **TF-iDF** препроцессингом
- Наименьшее значение **F1-score** ``0.82`` показал **RandomForestClassifier** классификатор с **TF-iDF** препроцессингом
- Наименьшее значение **Accuracy** ``0.8198666666666666`` показал **RandomForestClassifier** классификатор с **TF-iDF** препроцессингом